In [3]:
# 0. Settings
!pip install laion-clap
!pip install torch
!pip install torchvision
!pip install torchaudio
import laion_clap
import glob
import json
import torch
import numpy as np
import torchaudio
import tarfile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
import os
import glob
import tensorflow as tf
from tensorflow import keras
import pickle

In [2]:
class_index_dict_path = 'label/drumsc.json'
column_name = 'Instrument' # 'genre', 'artist'

# ESC50

In [ ]:
# 1. Path for audio/text files
zip_files_path = 'data/dtrain.zip'
unzip_files_folder_path = './data/dtrain'
upzip_audio_files_path = 'data/dtrain/**/*.wav'
upzip_text_files_path = 'data/dtrain/**/*.json'

# with zipfile.ZipFile(zip_files_path) as zip:
#     zip.extractall(unzip_files_folder_path)
audio_files = sorted(glob.glob(upzip_audio_files_path, recursive=True))
json_files = sorted(glob.glob(upzip_text_files_path, recursive=True)) 
class_index_dict = {k:v for k, v in json.load(open(class_index_dict_path)).items()}
ground_truth_idx = [class_index_dict[json.load(open(jf))[column_name][0]] for jf in json_files]

# 2. Pretrained Model calling
model = laion_clap.CLAP_Module(enable_fusion=True)
model.load_ckpt()

# 3. Get the Embeddings
# audio_embeds
BATCH_SIZE = 64
num_batches = len(audio_files) // BATCH_SIZE 
audio_embeds = []
for n in range(num_batches):
    curr_files = audio_files[n*BATCH_SIZE:(n+1)*BATCH_SIZE]
    audio_embeds.extend(model.get_audio_embedding_from_filelist(x=curr_files, use_tensor=False))
audio_embeds = np.array(audio_embeds)
# text_embeds
with torch.no_grad():
    all_texts = ["This song is " + t for t in class_index_dict.keys()]
    text_embeds = model.get_text_embedding(all_texts)

# drumset

# train dataset

In [3]:
# 1. Path for audio/text files
zip_files_path = 'data/dtrain.zip'
unzip_files_folder_path = './data/dtrain'
upzip_audio_files_path = 'data/dtrain/**/*.wav'
upzip_text_files_path = 'data/dtrain/**/*.json'

# with zipfile.ZipFile(zip_files_path) as zip:
#     zip.extractall(unzip_files_folder_path)
audio_files = sorted(glob.glob(upzip_audio_files_path, recursive=True))
json_files = sorted(glob.glob(upzip_text_files_path, recursive=True)) 
class_index_dict = {k:v for k, v in json.load(open(class_index_dict_path)).items()}
ground_truth_idx = [class_index_dict[json.load(open(jf))[column_name][0]] for jf in json_files]

# 2. Pretrained Model calling
model = laion_clap.CLAP_Module(enable_fusion=True)
model.load_ckpt()

# 3. Get the Embeddings
# audio_embeds
BATCH_SIZE = 64
num_batches = len(audio_files) // BATCH_SIZE 
audio_embeds = []
for n in range(num_batches):
    curr_files = audio_files[n*BATCH_SIZE:(n+1)*BATCH_SIZE]
    audio_embeds.extend(model.get_audio_embedding_from_filelist(x=curr_files, use_tensor=False))
audio_embeds = np.array(audio_embeds)
# text_embeds
with torch.no_grad():
    all_texts = ["This song is " + t for t in class_index_dict.keys()]
    text_embeds = model.get_text_embedding(all_texts)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [4]:
print(audio_embeds.shape)
print(len(audio_files))

(1408, 512)
1408


In [5]:
print(len(ground_truth_idx))

1408


In [6]:
print(audio_embeds.shape)
with open('data/dtrain_audio_embeds.pkl', 'wb') as f:
    pickle.dump(audio_embeds, f)
with open('data/dtrain_audio_embeds.pkl', 'rb') as f:
    audio_embeds = pickle.load(f)
print(audio_embeds.shape)

(1408, 512)
(1408, 512)


In [7]:
print(len(ground_truth_idx))
with open('data/dtrain_ground_truth_idx.pkl', 'wb') as f:
    pickle.dump((ground_truth_idx), f)
with open('data/dtrain_ground_truth_idx.pkl', 'rb') as f:
    ground_truth_idx = pickle.load(f)
print(len(ground_truth_idx))

1408
1408


# test dataset

In [1]:
# 1. Path for audio/text files
zip_files_path = 'data/dtest.zip'
unzip_files_folder_path = './data/dtest'
upzip_audio_files_path = 'data/dtest/**/*.wav'
upzip_text_files_path = 'data/dtest/**/*.json'

with zipfile.ZipFile(zip_files_path) as zip:
    zip.extractall(unzip_files_folder_path)
audio_files = sorted(glob.glob(upzip_audio_files_path, recursive=True))
json_files = sorted(glob.glob(upzip_text_files_path, recursive=True)) 
class_index_dict = {k:v for k, v in json.load(open(class_index_dict_path)).items()}
ground_truth_idx = [class_index_dict[json.load(open(jf))[column_name][0]] for jf in json_files]

NameError: name 'zipfile' is not defined

In [20]:
print(len(audio_files),len(json_files),len(ground_truth_idx))

30 30 30


In [21]:
# 2. Pretrained Model calling
model = laion_clap.CLAP_Module(enable_fusion=True)
model.load_ckpt()

# 3. Get the Embeddings
# audio_embeds
# BATCH_SIZE = 16  # 32에서 조정
# num_batches = len(audio_files) // BATCH_SIZE 
audio_embeds = model.get_audio_embedding_from_filelist(x=audio_files) 
audio_embeds = np.array(audio_embeds)
# # text_embeds
# with torch.no_grad():
#     all_texts = ["This song is " + t for t in class_index_dict.keys()]
#     text_embeds = model.get_text_embedding(all_texts)
    
# # 4. Save as pickles
# with open('data/drum_drumtest60_audio_embeds.pkl', 'wb') as f:
#     pickle.dump((audio_embeds), f)
# with open('data/drum_drumtest60_text_embeds.pkl', 'wb') as f:
#     pickle.dump((text_embeds), f)
# with open('data/drum_drumtest60_ground_truth_idx.pkl', 'wb') as f:
#     pickle.dump((ground_truth_idx), f)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [22]:
len(audio_embeds)

30

In [23]:
print(audio_embeds.shape)
with open('data/dtest_audio_embeds.pkl', 'wb') as f:
    pickle.dump(audio_embeds, f)
with open('data/dtest_audio_embeds.pkl', 'rb') as f:
    audio_embeds = pickle.load(f)
print(audio_embeds.shape)

(30, 512)
(30, 512)


In [24]:
print(len(ground_truth_idx))
with open('data/dtest_ground_truth_idx.pkl', 'wb') as f:
    pickle.dump((ground_truth_idx), f)
with open('data/dtest_ground_truth_idx.pkl', 'rb') as f:
    ground_truth_idx = pickle.load(f)
print(len(ground_truth_idx))

30
30


# Synthset

In [9]:
class_index_dict_path = 'label/synthsc.json'
column_name = 'Instrument' # 'genre', 'artist'

# train dataset

In [14]:
# 1. Path for audio/text files
zip_files_path = 'data/strain.zip'
unzip_files_folder_path = './data/strain'
upzip_audio_files_path = 'data/strain/**/*.wav'
upzip_text_files_path = 'data/strain/**/*.json'

# with zipfile.ZipFile(zip_files_path) as zip:
#     zip.extractall(unzip_files_folder_path)
audio_files = sorted(glob.glob(upzip_audio_files_path, recursive=True))
json_files = sorted(glob.glob(upzip_text_files_path, recursive=True)) 
class_index_dict = {k:v for k, v in json.load(open(class_index_dict_path)).items()}
ground_truth_idx = [class_index_dict[json.load(open(jf))[column_name][0]] for jf in json_files]

# 2. Pretrained Model calling
model = laion_clap.CLAP_Module(enable_fusion=True)
model.load_ckpt()

# 3. Get the Embeddings
# audio_embeds
BATCH_SIZE = 32
num_batches = len(audio_files) // BATCH_SIZE 
audio_embeds = []
for n in range(num_batches):
    curr_files = audio_files[n*BATCH_SIZE:(n+1)*BATCH_SIZE]
    audio_embeds.extend(model.get_audio_embedding_from_filelist(x=curr_files, use_tensor=False))
audio_embeds = np.array(audio_embeds)
# # text_embeds
# with torch.no_grad():
#     all_texts = ["This song is " + t for t in class_index_dict.keys()]
#     text_embeds = model.get_text_embedding(all_texts)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [15]:
print(audio_embeds.shape)
print(len(audio_files))

(288, 512)
288


In [16]:
print(audio_embeds.shape)
with open('data/strain_audio_embeds.pkl', 'wb') as f:
    pickle.dump(audio_embeds, f)
with open('data/strain_audio_embeds.pkl', 'rb') as f:
    audio_embeds = pickle.load(f)
print(audio_embeds.shape)

(288, 512)
(288, 512)


In [17]:
print(len(ground_truth_idx))
with open('data/strain_ground_truth_idx.pkl', 'wb') as f:
    pickle.dump((ground_truth_idx), f)
with open('data/strain_ground_truth_idx.pkl', 'rb') as f:
    ground_truth_idx = pickle.load(f)
print(len(ground_truth_idx))

288
288


# test dataset

In [18]:
# 1. Path for audio/text files
zip_files_path = 'data/stest.zip'
unzip_files_folder_path = './data/stest'
upzip_audio_files_path = 'data/stest/**/*.wav'
upzip_text_files_path = 'data/stest/**/*.json'

with zipfile.ZipFile(zip_files_path) as zip:
    zip.extractall(unzip_files_folder_path)
audio_files = sorted(glob.glob(upzip_audio_files_path, recursive=True))
json_files = sorted(glob.glob(upzip_text_files_path, recursive=True)) 
class_index_dict = {k:v for k, v in json.load(open(class_index_dict_path)).items()}
ground_truth_idx = [class_index_dict[json.load(open(jf))[column_name][0]] for jf in json_files]

In [19]:
# 2. Pretrained Model calling
model = laion_clap.CLAP_Module(enable_fusion=True)
model.load_ckpt()

# 3. Get the Embeddings
# audio_embeds
# BATCH_SIZE = 16  # 32에서 조정
# num_batches = len(audio_files) // BATCH_SIZE 
audio_embeds = model.get_audio_embedding_from_filelist(x=audio_files) 
audio_embeds = np.array(audio_embeds)
# # text_embeds
# with torch.no_grad():
#     all_texts = ["This song is " + t for t in class_index_dict.keys()]
#     text_embeds = model.get_text_embedding(all_texts)
    
# # 4. Save as pickles
# with open('data/drum_drumtest60_audio_embeds.pkl', 'wb') as f:
#     pickle.dump((audio_embeds), f)
# with open('data/drum_drumtest60_text_embeds.pkl', 'wb') as f:
#     pickle.dump((text_embeds), f)
# with open('data/drum_drumtest60_ground_truth_idx.pkl', 'wb') as f:
#     pickle.dump((ground_truth_idx), f)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [20]:
len(audio_embeds)

31

In [21]:
print(audio_embeds.shape)
with open('data/stest_audio_embeds.pkl', 'wb') as f:
    pickle.dump(audio_embeds, f)
with open('data/stest_audio_embeds.pkl', 'rb') as f:
    audio_embeds = pickle.load(f)
print(audio_embeds.shape)

(31, 512)
(31, 512)


In [22]:
print(len(ground_truth_idx))
with open('data/stest_ground_truth_idx.pkl', 'wb') as f:
    pickle.dump((ground_truth_idx), f)
with open('data/stest_ground_truth_idx.pkl', 'rb') as f:
    ground_truth_idx = pickle.load(f)
print(len(ground_truth_idx))

31
31
